<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2024/ML2024_ex05notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex05notebookA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2024)


----
## ロジスティック回帰＋勾配法によるパラメータの最適化 (3)
----




----
### 準備


以下，コードセルを上から順に実行してながら読んでいってね．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc  # アニメーションのため
import pandas as pd
import seaborn
seaborn.set()

----
### 最急降下法によるロジスティック回帰モデルの学習（2クラス識別の場合）

2クラス識別のロジスティック回帰モデルで，パラメータを最急降下法で最適化する手順を考えましょう．

2クラス識別のロジスティック回帰の問題設定は以下の通りでした．


**［ロジスティック回帰の問題設定（2クラスの場合）］**

$D$次元のデータを二つのクラスに識別するモデルを学習させる．学習データは $N$ 個あり，次のように与えられる．

$$
(\mathbf{x}_1, y_1), (\mathbf{x}_2, y_2),\ldots , (\mathbf{x}_N, y_N)
$$



ただし，$\mathbf{x}_n \in {\cal R}^{D}$ はモデルへの入力であり，$y_n \in \{0, 1\}$ はこのデータの所属クラスの正解を表す値である（$n=1,2,\ldots,N$）．

学習モデルは次式で定める．

$$
\begin{aligned}
f(\mathbf{x}) &= \sigma\left( w_0 + \sum_{d=1}^{D}w_dx_d \right) \\
 &= \frac{1}{\displaystyle 1+\exp{\left( - \left( w_0 + \sum_{d=1}^{D}w_dx_d \right) \right)}} \qquad (1)
\end{aligned}
$$

ここで， $\sigma(s)$ はシグモイド関数 $\sigma(s) = \frac{1}{1+e^{-s}}$ を表す．
このモデルのパラメータは $w_0, w_1, \ldots, w_D$ の $(D+1)$ 個ある．

このとき，モデルの出力と正解の値との間の「遠さ」を，次式の交差エントロピーで定義する．
$$
\begin{aligned}
H &= -\sum_{n=1}^{N} \left( y_n\log{f(\mathbf{x}_n})+(1-y_n)\log{\left( 1-f(\mathbf{x}_n)\right)} \right) \qquad (2)
\end{aligned}
$$
この $H$ の値がなるべく小さくなるようにパラメータ $w_0, w_1, \ldots, w_D$ を求めたい．

パラメータをならべたベクトルを $\mathbf{w} = (w_0, w_1, \ldots, w_D)$ と表すことにして，$H$ を最小にする $\mathbf{w}$ を求める最急降下法の手続きを導出します．

#### 勾配の計算

最急降下法では $H$ の $\mathbf{w}$ に関する勾配
$$
\nabla{H} = \left( \frac{\partial H}{\partial w_0}, \frac{\partial H}{\partial w_1}, \ldots, \frac{\partial H}{\partial w_D}\right)
$$
が必要ですので，$\frac{\partial H}{\partial w_d}$ ($d=0,1,\ldots,D$) を求めましょう．

まず，

$$
\ell_n = y_n\log f(\mathbf{x}_n) + (1-y_n)\log(1-f(\mathbf{x}_n)) \qquad (3)\\
$$

とおくことにします．このとき，

$$
\frac{\partial H}{\partial w_d} = -\sum_{n=1}^{N}\frac{\partial \ell_n}{\partial w_d} \qquad (4)
$$

です． $\frac{\partial \ell_n}{\partial w_d}$ を計算すると


$$
\frac{\partial \ell_n}{\partial w_d} = \left(y_n-f(\mathbf{x}_n)\right)x_{n,d} \qquad (n = 1, 2, \ldots, N, d = 0, 1, \ldots, D) \qquad (5)
$$

が得られます（notebookC参照）．ただし，$x_{n,0} \equiv 1$ としました．



以上より，勾配ベクトルの要素は

$$
\begin{aligned}
\frac{\partial H}{\partial w_d} &= -\sum_{n=1}^{N}\frac{\partial \ell_n}{\partial w_d} = -\sum_{n}^{N}(y_n - f(\mathbf{x}_n))x_{n,d}\qquad (d = 0, 1, \ldots, D) \qquad (6)
\end{aligned}
$$

となります．ベクトルの形にまとめると，

$$
\nabla{H} =  -\sum_{n}^{N}(y_n - f(\mathbf{x}_n)) \mathbf{x}_n \qquad (7)
$$

と書けます．

#### パラメータ更新式の導出

勾配の式が求まりました．これを用いて最急降下法によるパラメータ更新式を求めると，次のようになります．

$$
\begin{aligned}
\mathbf{w}^{\rm new} &= \mathbf{w} - \eta \nabla H \\
&=\mathbf{w} + \eta\sum_{n}^{N}(y_n - f(\mathbf{x}_n)) \mathbf{x}_n \qquad (8)
\end{aligned}
$$

$\eta$ は正の定数（学習係数）です．

交差エントロピーやシグモイドに $\log$ や $\exp$ が入っていたわりには，パラメータを更新するための式はシンプルな形になります．
$f(\mathbf{x}_n)$ はデータ $\mathbf{x}_n$ をモデルに入力して得られる出力（$0 < f(\mathbf{x}_n) < 1$），$y_n$ はその正解の値（$y_n \in\{0,1\}$）でした．
この式を見ると，両者の差 $y_n - f(\mathbf{x}_n)$ と入力の値との積に応じてパラメータを更新することになっています．

----
### 例: 2次元2クラスのデータのロジスティック回帰

最急降下法のパラメータ更新式が求まりましたので，ロジスティック回帰モデルの学習手順をプログラムとして書くことができます．
以前使ったのと同じ2次元2クラスのデータで実際にロジスティック回帰の学習を行ってみましょう．

#### データの準備

In [ ]:
## 2次元正規分布で2クラスまたは3クラスのデータを生成する関数

def getData(nclass=2, seed = None):

    assert nclass == 2 or nclass == 3

    if seed != None:
        np.random.seed( seed )

    # 2次元の spherical な正規分布3つからデータを生成
    X0 = 1.0*np.random.randn(200, 2) + [3.0, 3.0]
    X1 = 1.0*np.random.randn(200, 2) + [7.0, 6.0]
    X2 = 0.5*np.random.randn(200, 2) + [3.0, 7.0]

    # それらのラベル用のarray
    lab0 = np.zeros(X0.shape[0], dtype = int)
    lab1 = np.zeros(X1.shape[0], dtype = int) + 1
    lab2 = np.zeros(X2.shape[0], dtype = int) + 2

    # X （入力データ）, label （クラスラベル）, y（教師信号） をつくる
    if nclass == 2:
        X = np.vstack((X0, X1))
        label = np.hstack((lab0, lab1))
        y = np.zeros(X.shape[0])
        y[label == 1] = 1.0
    else:
        X = np.vstack((X0, X1, X2))
        label = np.hstack((lab0, lab1, lab2))
        y = np.zeros((X.shape[0], nclass))
        for ik in range(nclass):
            y[label == ik, ik] = 1.0

    return X, label, y

In [ ]:
# データの準備
X2c, lab2c, y2c = getData(nclass=2, seed=0)
N, D = X2c.shape
X2c = np.vstack((np.ones(N), X2c.T)).T
print(f'データ数 N = {N}, 次元数 D = {D}')

fig = plt.figure(facecolor='white', figsize=(14, 6))

# 左の2次元散布図
ax0 = fig.add_subplot(121)
ax0.set_xlim(0, 10)
ax0.set_ylim(0, 10)
ax0.set_aspect('equal')
ax0.scatter(X2c[y2c == 0, 1], X2c[y2c == 0, 2]) # blue
ax0.scatter(X2c[y2c == 1, 1], X2c[y2c == 1, 2]) # orange
ax0.set_xlabel('$x_1$')
ax0.set_ylabel('$x_2$')

# 右の3次元散布図
elevation = 20
azimuth = -70
ax1 = fig.add_subplot(122, projection='3d')
ax1.scatter(X2c[y2c==0, 1], X2c[y2c==0, 2], 1) # blue
ax1.scatter(X2c[y2c==1, 1], X2c[y2c==1, 2], 2) # orange
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 10)
ax1.view_init(elevation, azimuth)
ax1.set_xlabel('$x_1$')
ax1.set_ylabel('$x_2$')
ax1.set_zlabel('$y$')

plt.show()

#### 学習


次のセルは，学習のための関数の定義です．

In [ ]:
# モデル出力の計算
def model2(X, w):
    return 1.0 / (1.0 + np.exp(-(X @ w)))

# 交差エントロピーと正解数
def score2(yt, y):
    ce = -np.sum(y*np.log(yt)+(1.0-y)*np.log(1.0-yt)) # 交差エントロピー
    count = np.sum((yt >= 0.5)*y) + np.sum((yt < 0.5)*(1 - y)) # 正解数
    return ce, count

# 勾配の計算
def grad2(X, yt, y):
    return (yt - y) @ X

次のセルを実行すると，上記の2次元2クラスデータのロジスティック回帰モデルの学習を行います．

In [ ]:
# パラメータの初期化
w = (np.random.random(D+1) - 0.5) * 0.2 # [-0.1, 0.1) の一様乱数

# 学習係数と学習繰り返し回数
eta = 0.2/N
nitr = 1000

# 学習
for i in range(nitr+1):
    yt = model2(X2c, w)     # モデル出力の計算
    ce, count = score2(yt, y2c) # 交差エントロピーと正解数の計算
    dw = grad2(X2c, yt, y2c) # 勾配の計算
    w -= eta * dw       # パラメータの更新
    if (i < 100 and i % 10 == 0) or (i % 100 == 0):
        print(f'{i}  {ce:.4f}  {float(count)/N}')

出力される数値は，左から順に「学習繰り返し回数」，「（学習データに対する）交差エントロピー」，「（同）識別率」です．最急降下法のステップを繰り返すごとに，交差エントロピーの値が減少していること，それに連れて識別率は上昇していることが分かります．

学習（アニメーション版）

以下のセルを実行すると，上記と同様の学習の過程をアニメーションに描きます．

In [ ]:
# パラメータの初期化
w = (np.random.random(D+1) - 0.5) * 0.2 # [-0.1, 0.1) の一様乱数

# 学習係数と学習繰り返し回数
eta = 0.2/N
nitr = 1000

fig = plt.figure(facecolor='white', figsize=(12, 6))
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122)
elevation = 20
azimuth = -70
ax1.view_init(elevation, azimuth)
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 10)
ax1.set_zlim(0, 1)
ax1.scatter(X2c[y2c==0, 1], X2c[y2c==0, 2], 0)
ax1.scatter(X2c[y2c==1, 1], X2c[y2c==1, 2], 1)
#fig.show()

ax2.set_xlim(0, nitr)
ax2.set_ylim(0, 300)

aList = []
xx, yy = np.meshgrid(np.linspace(0, 10, num=16), np.linspace(0, 10, num=16))
xxr, yyr = xx.ravel(), yy.ravel()
XX = np.vstack((np.ones(xxr.shape[0]), xxr, yyr)).T

iList = []
ceList = []

for i in range(nitr+1):

    yt = model2(X2c, w)     # モデル出力の計算
    ce, count = score2(yt, y2c) # 交差エントロピーと正解数の計算
    dw = grad2(X2c, yt, y2c) # 勾配の計算
    w -= eta * dw       # パラメータの更新

    if (i < 100 and i % 10 == 0) or i % 100 == 0:
        iList.append(i)
        ceList.append(ce)
        ZZ = model2(XX, w)
        zz = ZZ.reshape(xx.shape)
        a1 = ax1.plot_wireframe(xx, yy, zz, color='green')
        a2 = ax2.plot(iList, ceList, color='blue', marker='.')
        rr = count/N*100
        s = f'H = {ce:.3f}\nrate = {rr:.1f}%'
        a3 = ax2.text(500, 240, s, size=20)
        aList.append([a1]+a2 + [a3])

anim = animation.ArtistAnimation(fig, aList, interval=300)
rc('animation', html='jshtml')
plt.close()
anim


上記のセルは実行するたびにパラメータが異なる乱数で初期化されます．何度かセルを実行して，どんな違いが出るか確認してみましょう．

----
### 一般のロジスティック回帰モデルとその学習


ここまではクラス数が2の場合限定でロジスティック回帰の話をしてきました．ここからは，クラス数が3以上の問題でも使える一般のロジスティック回帰モデルとその学習について説明します．
以下，クラス数を文字 $K$ で表します．

#### 一般のロジスティック回帰モデル


**［ロジスティック回帰の問題設定（$K$クラスの場合）］**

$D$次元のデータを$K$個のクラスに識別するモデルを学習させる．学習データは $N$ 個あり，次のように与えられる．

$$
(\mathbf{x}_1, \mathbf{y}_1), (\mathbf{x}_2, \mathbf{y}_2),\ldots , (\mathbf{x}_N, \mathbf{y}_N)
$$

ただし，$\mathbf{x}_n \in {\cal R}^{D}$ はモデルへの入力である．また，$\mathbf{y}_n \in \{0, 1\}^{K}$ （$0$か$1$のみを要素にもつ$K$次元ベクトル）はこのデータの所属クラスの正解を表す値である（$n=1,2,\ldots,N$）．

$\mathbf{y}_n = (y_{n,1}, y_{n,2}, \ldots, y_{n,K})$ の要素 $y_{n,k}$ は，$n$番目の学習データが $k$ 番目のクラスに所属するならば $1$，さもなくば $1$ をとる．したがって，$\mathbf{y}_n$ の要素はどれか一つだけが $1$ で他は全て $0$ である．

学習モデルは次式で定める．
$$
\begin{aligned}
\hat{y}_k &= \frac{\exp s_k}{\displaystyle\sum_{k=1}^{K}\exp{s_k}} \qquad (k = 1, 2, \ldots, K) \qquad (10)\\
s_k &= w_{k,0} + \sum_{d=1}^{D}w_{k,d}x_d
\end{aligned}
$$
このモデルのパラメータは $w_{k,d}$ ($k = 1, 2, \ldots, K, d = 0, 1, \ldots, D$) の $K\times (D+1)$ 個ある．

このとき，モデルの出力と正解の値との間の「遠さ」を，次式の交差エントロピーで定義する．
$$
\begin{aligned}
H &= -\sum_{n=1}^{N} \sum_{k=1}^{K} y_{n,k}\log{\hat{y}_{n,k}}
\end{aligned}
$$
この $H$ の値がなるべく小さくなるようにパラメータ $\{ w_{k,d} \}$ を求めたい．

2クラス識別のロジスティック回帰モデルの出力は1つでしたが，一般化したロジスティック回帰モデルでは，$\hat{y}_1,\hat{y}_2,\ldots, \hat{y}_K$ と $K (=\mbox{クラス数})$ 個あります．大雑把にいうと，2クラス識別のロジスティック回帰モデルを $K$ 個あわせたものになっています．

ただし，2クラス識別では出力の値をシグモイド関数によって計算していたところが，こちらでは式(10)のようになっています．
この式(10)は，**ソフトマックス関数**（**softmax**関数）と呼ばれるものです．
式の形から明らかですが，$0 < \hat{y}_k < 1$ および $\sum_{k=1}^{K}\hat{y}_k = 1$ となる性質があります．

このことから，ある入力データに対するモデル出力 $\hat{y}_1,\hat{y}_2,\ldots, \hat{y}_K$ の値は，そのデータがそれぞれのクラスに所属する確信度合い（確率）を表すと解釈することができます．
上記の交差エントロピーを最小化することで，正解クラスに対応する出力が $1$ に近づき，それ以外のクラスに対応する出力が $0$ に近づくようになります．

#### 一般のロジスティック回帰モデルの学習

導出過程は省略しますが，上記の交差エントロピのパラメータに関する勾配は次のようになります．

$$
\frac{\partial H}{\partial w_{k,d}} =  -\sum_{n}^{N}(y_{n,k} - \hat{y}_{n,k}) x_{n,d} \qquad(k = 1, 2, \ldots, K, d = 0, 1, \ldots, D)
$$


2クラスの場合と同じ構造をしていますね．

以下，最急降下法の手順は2クラスの場合とほとんど同じですので，説明を省略します．

----
### 例: 2次元3クラスのデータのロジスティック回帰

2次元のデータを3クラスに識別する問題にロジスティック回帰を適用してみましょう．

#### データの準備

In [ ]:
# データの準備
K = 3
X3c, lab3c, Y3c = getData(nclass=K, seed=0)
N, D = X3c.shape
X3c = np.vstack((np.ones(N), X3c.T)).T
print(f'データ数 N = {N}, 次元数 D = {D}')

fig = plt.figure(facecolor='white', figsize=(14, 6))

# 2次元散布図
ax0 = fig.add_subplot(111)
ax0.set_xlim(0, 10)
ax0.set_ylim(0, 10)
ax0.set_aspect('equal')
ax0.scatter(X3c[lab3c == 0, 1], X3c[lab3c == 0, 2]) # blue
ax0.scatter(X3c[lab3c == 1, 1], X3c[lab3c == 1, 2]) # orange
ax0.scatter(X3c[lab3c == 2, 1], X3c[lab3c == 2, 2]) # green
ax0.set_xlabel('$x_1$')
ax0.set_ylabel('$x_2$')

plt.show()

#### 学習



次のセルは，学習のための関数の定義です．

In [ ]:
# モデル出力の計算
def modelK(X, W):
    s = np.exp(X @ W.T)
    Yt = s / np.sum(s, axis=1)[:, np.newaxis]
    return Yt

# 交差エントロピーと正解数
def scoreK(Y, Yt):
    ce = -np.sum(Y * np.log(Yt))/len(Y)
    count = np.sum(np.argmax(Y, axis=1) == np.argmax(Yt, axis=1))
    return ce, count

# 勾配の計算
def gradK(X, Y, Yt):
    return (Yt - Y).T @ X

次のセルを実行すると，上記の2次元3クラスデータのロジスティック回帰モデルの学習を行います．

In [ ]:
# パラメータの初期化
W = (np.random.random((K, D+1)) - 0.5) * 0.2 # [-0.1, 0.1) の一様乱数

# 学習係数と学習繰り返し回数
eta = 0.2/N
nitr = 1000

# 学習
for i in range(nitr+1):
    Yt = modelK(X3c, W)        # モデル出力の計算
    ce, count = scoreK(Y3c, Yt) # 交差エントロピーと正解数の計算
    dW = gradK(X3c, Y3c, Yt)   # 勾配の計算
    W -= eta * dW              # パラメータの更新
    if (i < 100 and i % 10 == 0) or (i % 100 == 0):
        print(f'{i}  {ce:.4f}  {float(count)/N}')

学習を繰り返すにつれて交差エントロピーが減少し，学習データに対する正答率（正しく識別できたものの割合）が上昇していることが分かります．

次のセルを実行すると，学習によって得られたモデルが2次元平面をどのように3つのクラスに分類しているかを可視化させることができます．

In [ ]:
# グラフ描画用のグリッドデータの作成
xmin, xmax = 0, 10
ymin, ymax = 0, 10
npoints = 100
dx, dy = (xmax - xmin)/npoints, (ymax - ymin)/npoints
x_mesh, y_mesh = np.mgrid[xmin:xmax:dx, ymin:ymax:dy]
X_mesh = np.dstack((x_mesh, y_mesh))

# X_mesh の各点における事後確率の推定
XX = X_mesh.reshape((-1, 2))
XX = np.vstack((np.ones(len(XX)), XX.T)).T
Yt = modelK(XX, W)
pp = Yt.reshape((X_mesh.shape[0], X_mesh.shape[1], K))

# グラフの描画
fig, ax = plt.subplots()
cmap = ['Blues', 'Oranges', 'Greens']
for k in range(K):
    ax.scatter(X3c[lab3c == k, 1], X3c[lab3c == k, 2])
    ax.contourf(x_mesh, y_mesh, pp[:, :, k], levels=[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect('equal')
plt.show()

図で3色に塗られた領域の色の濃さは，その位置に対応する値をモデルに入力したときのモデル出力のうち，その色に対応する値の大きさを表しています．上述のように，$K$個のモデル出力は，入力データがそれぞれのクラスに属する確率の推定値を表しています．
図を見ると，学習データの点の多くが正しく塗り分けられていることが分かります．